<a href="https://colab.research.google.com/github/hsandaver/essays/blob/main/DyeClassifierLABv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚀 Enhanced and Fancy LAB Color Analyzer with Dye Integration 🚀

# ------------------------------
# 1. Install and Import Libraries
# ------------------------------

import sys
import subprocess

def install_packages(packages):
    """
    Installs the listed packages using pip if they are not already installed.
    """
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = ['colormath', 'plotly', 'ipywidgets', 'pandas', 'numpy']
install_packages(required_packages)

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from google.colab import files
import io
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Enable ipywidgets in Colab
from google.colab import output
output.enable_custom_widget_manager()

# ------------------------------
# 2. Define Utility Functions
# ------------------------------

def upload_dataset(expected_filenames):
    """
    Prompts the user to upload files interactively, then allows selection of the correct file if the expected filename isn't found.
    """
    print(f"📂 Please upload the required file(s).")
    uploaded = files.upload()

    # List all uploaded files
    uploaded_files = list(uploaded.keys())
    print(f"📂 Uploaded Files: {uploaded_files}")

    # Check for the expected filenames
    for expected_filename in expected_filenames:
        if expected_filename in uploaded_files:
            df = pd.read_csv(io.BytesIO(uploaded[expected_filename]))
            print(f"✅ '{expected_filename}' uploaded successfully.")
            return df
    else:
        # If expected files are not found, prompt user to select one
        print(f"❌ None of the expected files ({expected_filenames}) were found.")
        print("🔍 Please select one of the uploaded CSV files:")
        for idx, file in enumerate(uploaded_files, start=1):
            print(f"{idx}. {file}")

        # Prompt user to input the number corresponding to their file
        try:
            selected_idx = int(input("Enter the number of the file to use: ")) - 1
            if selected_idx < 0 or selected_idx >= len(uploaded_files):
                raise ValueError("Invalid selection.")
        except (ValueError, IndexError):
            raise ValueError("Invalid selection. Please restart the process and select a valid file.")

        selected_file = uploaded_files[selected_idx]
        df = pd.read_csv(io.BytesIO(uploaded[selected_file]))
        print(f"✅ '{selected_file}' uploaded successfully.")
        return df

def validate_dataset(df):
    """
    Validates that the DataFrame contains the necessary columns and no missing values.
    """
    required_columns = ['L', 'A', 'B', 'Color Name']
    missing = set(required_columns) - set(df.columns)
    if missing:
        raise ValueError(f"CSV file is missing required columns: {missing}")

    df['L'] = pd.to_numeric(df['L'], errors='coerce')
    df['A'] = pd.to_numeric(df['A'], errors='coerce')
    df['B'] = pd.to_numeric(df['B'], errors='coerce')

    if df[['L', 'A', 'B']].isnull().values.any():
        raise ValueError("Missing or invalid numeric values in 'L', 'A', or 'B' columns.")

def calculate_delta_e(input_lab, dataset_df):
    """
    Vectorized Delta-E (CIE76) calculation between input LAB and dataset LAB colors.
    """
    delta_e = np.linalg.norm(dataset_df[['L', 'A', 'B']].values - input_lab, axis=1)
    return delta_e

def find_closest_color(input_lab, dataset_df):
    """
    Finds the closest color in the dataset to the input LAB color based on Delta-E.
    """
    delta_e_values = calculate_delta_e(input_lab, dataset_df)
    min_idx = np.nanargmin(delta_e_values)
    min_delta_e = delta_e_values[min_idx]
    closest_color = dataset_df.iloc[min_idx]
    return closest_color, min_delta_e

def lab_to_rgb(lab_color):
    """
    Converts a LAB color to RGB.
    """
    try:
        lab = LabColor(lab_l=lab_color[0], lab_a=lab_color[1], lab_b=lab_color[2])
        rgb = convert_color(lab, sRGBColor, target_illuminant='d65')
        rgb_clamped = (
            int(max(0, min(rgb.rgb_r, 1)) * 255),
            int(max(0, min(rgb.rgb_g, 1)) * 255),
            int(max(0, min(rgb.rgb_b, 1)) * 255)
        )
        return rgb_clamped
    except Exception as e:
        print(f"❌ Error converting LAB to RGB: {e}")
        return (0, 0, 0)

def validate_lab_color(lab):
    """
    Validates the input LAB color.
    """
    if not isinstance(lab, (list, tuple, np.ndarray)) or len(lab) != 3:
        raise ValueError("Input LAB color must be a list, tuple, or array of three numerical values.")
    L, A, B = lab
    if not (0 <= L <= 100) or not (-128 <= A <= 127) or not (-128 <= B <= 127):
        raise ValueError("L, A, and B components are out of bounds.")

# ------------------------------
# 3. Dye Integration Functions
# ------------------------------

# Mapping estimated color descriptions to rough LAB values
dye_color_lab_map = {
    "red": [60, 70, 30],  # LAB for red
    "blue": [32, 5, -60],  # LAB for blue
    "yellow": [97, -21, 94],  # LAB for yellow
    "green": [46, -50, 50],  # LAB for green
    "orange": [75, 23, 78]  # LAB for orange
}

def match_dye_to_lab(input_lab, dye_colors_df, color_lab_map):
    """
    Matches the input LAB color to the closest dye based on Delta-E.
    """
    closest_dye = None
    min_delta_e = float('inf')

    for _, dye_row in dye_colors_df.iterrows():
        dye_color_desc = dye_row["Estimated Color"].lower()
        for color, lab_val in color_lab_map.items():
            if color in dye_color_desc:
                delta_e = np.linalg.norm(np.array(lab_val) - np.array(input_lab))
                if delta_e < min_delta_e:
                    min_delta_e = delta_e
                    closest_dye = {
                        "Dye Color Description": dye_color_desc,
                        "Dye Name": dye_row.get("Chromophore", "Unknown Dye"),
                        "Dye SMILES": dye_row.get("Corrected_SMILES", "Unknown"),
                        "Closest Dye LAB": lab_val,
                        "Delta-E": min_delta_e
                    }

    return closest_dye

def display_dye_info(closest_dye):
    """
    Displays information about the closest matching dye.
    """
    if closest_dye:
        print(f"🎨 Closest Dye Description: {closest_dye['Dye Color Description']}")
        print(f"🔬 Dye Chromophore: {closest_dye['Dye Name']}")
        print(f"🧪 Dye SMILES: {closest_dye['Dye SMILES']}")
        print(f"📏 Delta-E Value (to Dye): {closest_dye['Delta-E']:.2f}")
    else:
        print("❌ No matching dye found.")

# ------------------------------
# 4. Interactive User Interface
# ------------------------------

def run_color_analyzer_with_dyes():
    """
    Runs the interactive LAB Color Analyzer, including dye matching.
    """
    try:
        dataset_df = upload_dataset(['iscc_nbs_lab_colors.csv'])  # ISCC-NBS LAB colors
        dye_colors_df = upload_dataset(['output_dye_colors_enhanced_with_corrections(1).csv'])  # Dye dataset
        validate_dataset(dataset_df)
        print("✅ Datasets uploaded and validated successfully.")
    except Exception as e:
        print(f"❌ Error uploading or validating dataset: {e}")
        return

    # Interactive widgets for LAB input
    lab_l_slider = widgets.FloatSlider(value=50.0, min=0.0, max=100.0, step=0.01, description='L:', continuous_update=False)
    lab_a_slider = widgets.FloatSlider(value=0.0, min=-128.0, max=127.0, step=0.01, description='A:', continuous_update=False)
    lab_b_slider = widgets.FloatSlider(value=0.0, min=-128.0, max=127.0, step=0.01, description='B:', continuous_update=False)
    run_button = widgets.Button(description='🔍 Find Closest Color', button_style='success', icon='search')
    output = widgets.Output()

    display(widgets.VBox([lab_l_slider, lab_a_slider, lab_b_slider, run_button]), output)

    def on_run_button_clicked(b):
        with output:
            clear_output(wait=True)
            input_lab = [lab_l_slider.value, lab_a_slider.value, lab_b_slider.value]
            try:
                validate_lab_color(input_lab)
                print(f"🟢 Input LAB Color: L={input_lab[0]}, A={input_lab[1]}, B={input_lab[2]}")

                # Find closest ISCC-NBS color
                closest_color, delta_e = find_closest_color(np.array(input_lab), dataset_df)
                closest_color_name = closest_color['Color Name']
                closest_lab = [closest_color['L'], closest_color['A'], closest_color['B']]
                print(f"🎨 Closest ISCC-NBS Color: {closest_color_name}")
                print(f"📏 Delta-E Value: {delta_e:.2f}")

                # Convert to RGB
                input_rgb = lab_to_rgb(input_lab)
                closest_rgb = lab_to_rgb(closest_lab)
                print(f"🎨 Input RGB: {input_rgb}")
                print(f"🎨 Closest RGB: {closest_rgb}")

                # Find closest dye
                closest_dye = match_dye_to_lab(input_lab, dye_colors_df, dye_color_lab_map)
                display_dye_info(closest_dye)

            except ValueError as ve:
                print(f"❌ Input validation error: {ve}")
            except Exception as e:
                print(f"❌ An unexpected error occurred: {e}")

    run_button.on_click(on_run_button_clicked)

# Execute the analyzer with dye matching
run_color_analyzer_with_dyes()